## Data Preprocessing

in this document I'll be showing how I scraping/parsing images from stockx.com for my Machine Learning Nano Degree Project.

First I went look into a paticular shoe's product link and image link it loosk like this:

https://stockx.com/nike-air-max-1-97-sean-wotherspoon-na


![title](StockX_Prod.png)

Here's the image link:


https://stockx-360.imgix.net/Nike-Air-Max-1-97-Sean-Wotherspoon-NA/Images/Nike-Air-Max-1-97-Sean-Wotherspoon-NA/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1538080256&w=1300

So we can observe here the product link is constructed by the shoe name itself as the sublink, and also for the iamge link, in the folder it's in sequence for 0 to 360 degree image coded as img01.jpg to img35.jpg.

I'll use these two features to write my script for scraping images.

In [ ]:
from stockxsdk import Stockx
import pandas as pd

stockx = Stockx()

email = '***************'
password = '***************'
stockx.authenticate(email, password)

keyword=['Zoom Generation','Adidas NMD R1','Adidas Superstar','Adidas Campus','Adidas Yung 1','Adidas Prophere','Adidas Stan Smith',
         'Adidas Night Jogger','Adidas EQT','Adidas ZX500','Adidas Ozweego','Adidas Harden','Adidas Forum']

# Keyword for searching sneakers on stockx.com

resp=stockx.search('Yeezy')
r=pd.DataFrame(resp)

for k in keyword:
    t=pd.DataFrame(stockx.search(k))
    r=pd.concat([r,t])


Using stockxsdk (https://pypi.org/project/stockx-py-sdk/) search function here to generated list of product url by keywords

After cleaned invalid product (not shoe category) and 0 sales item, I have a clean csv file here (output.csv)

We'll be using label to create folder and url to parse the image in that category

In [98]:
df=pd.read_csv('output.csv')
df.head()

,brand,categories,colorway,deadstock_sold,gender,style_id,thumbnail_url,url,idx,label
0,adidas,"['adidas', 'Yeezy']",Clay/Clay/Clay,1560.0,men,EG7490,https://stockx.imgix.net/adidas-Yeezy-Boost-35...,adidas-yeezy-boost-350-v2-clay,0,Adidas-Yeezy-350-V2
1,adidas,"['adidas', 'Yeezy']",Grey/Grey/Grey,8542.0,men,EG7492,https://stockx.imgix.net/adidas-Yeezy-Boost-35...,adidas-yeezy-boost-350-v2-true-form,1,Adidas-Yeezy-350-V2
2,addias,"['adidas', 'Yeezy']",Geode/Geode/Geode,1718.0,men,EG6860,https://stockx.imgix.net/adidas-Yeezy-Boost-70...,adidas-yeezy-boost-700-v2-geode,2,Adidas-Yeezy-700-V2
3,adidas,"['adidas', 'Yeezy']",Grey/Grey/Grey,1284.0,men,EG7491,https://stockx.imgix.net/adidas-Yeezy-Boost-35...,adidas-yeezy-boost-350-v2-hyperspace,3,Adidas-Yeezy-350-V2
4,adidas,"['adidas', 'Yeezy']",Grey/Grey/Inertia,7665.0,men,EG7597,https://stockx.imgix.net/adidas-Yeezy-Boost-70...,adidas-yeezy-boost-700-inertia,4,Adidas-Yeezy-700


Define a helper function to create folder for different sneakers, the folder name will be using the name in the url

In [ ]:
import os
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)      

Using urllib and bs4 to scraping all the images, first we request product page, then getting image src link from bs4, using it to loop through img01 to img35, if it's not an .jpg file we put .png instead. The filename convention weill be the index in our csv file + angle (0-35)

In [ ]:
import urllib.request
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

for index, row in df.iterrows():  
    a=[s.capitalize() for s in s.split('/')[3].split('Product')[0].split('-')]
    createFolder()
    b='-'.join(a)[:-1]
    f='-'.join(a[0:3])
    createFolder('E:/StockX/'+f)
    temp_url='https://stockx.com/'+df["url"][index]
    result = requests.get(temp_url)
    c=result.content
    soup = BeautifulSoup(c,"lxml")
    try:
        url=soup.find_all("img")[12].get('src')
        if('img01' in url):
            url=url.split('?')[0]
            x,y=url.split('img01')
            print (x,y)
            for i in img_cd:
                try:
                    urllib.request.urlretrieve(x+'img'+i+y, 'E:/StockX/'+f+'/'+str(index)+'_'+i+'.jpg')
                except:
                    pass
        else:
            try:
                urllib.request.urlretrieve(url, 'E:/StockX/'+f+'/'+str(index)+'_'+i+'.png')
            except:
                pass
    except:
        pass


Now I'm doing some manual clean up, removing wrong category, combine duplicated folder.
As a result I'm having 52593 images with 179 folder (labels)


![title](Folder.png)

![title](Folder_2.png)

Now I'm using this helper function to seperate this dataset to train/test data with a 20% split

In [103]:
import os
import shutil

def split_dataset_into_test_and_train_sets(all_data_dir, training_data_dir, testing_data_dir, testing_data_pct):
    # Recreate testing and training directories
    if testing_data_dir.count('/') > 1:
        shutil.rmtree(testing_data_dir, ignore_errors=False)
        os.makedirs(testing_data_dir)
        print("Successfully cleaned directory " + testing_data_dir)
    else:
        print("Refusing to delete testing data directory " + testing_data_dir + " as we prevent you from doing stupid things!")

    if training_data_dir.count('/') > 1:
        shutil.rmtree(training_data_dir, ignore_errors=False)
        os.makedirs(training_data_dir)
        print("Successfully cleaned directory " + training_data_dir)
    else:
        print("Refusing to delete testing data directory " + training_data_dir + " as we prevent you from doing stupid things!")

    num_training_files = 0
    num_testing_files = 0

    for subdir, dirs, files in os.walk(all_data_dir):
        category_name = os.path.basename(subdir)

        # Don't create a subdirectory for the root directory
        print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
            continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name

        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)

        for file in files:
            input_file = os.path.join(subdir, file)
            if np.random.rand(1) < testing_data_pct:
                shutil.copy(input_file, testing_data_dir + '/' + category_name + '/' + file)
                num_testing_files += 1
            else:
                shutil.copy(input_file, training_data_dir + '/' + category_name + '/' + file)
                num_training_files += 1

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [109]:
split_dataset_into_test_and_train_sets('E:/StockX/','E:/StockX_Train/','E:/StockX_Test/',0.2)

![title](Split.png)

Now we wrapping up data processing portion here